In [ ]:
%matplotlib notebook
#%matplotlib inline

import matplotlib.pyplot as plt
import rasterio
import rasterio.warp
import numpy as np
import math
from scipy import ndimage as ndi

import pyproj
import xarray as xr
import fiona
import shapely
import pydash
from osgeo import osr
from numpy import pi, r_, c_, vstack, hstack
import sys

from kk.dggs import DGGS
from kk.tools import polygon_path, spread_num_samples, gen_pts_from_distribution

In [ ]:
def get_affine(fname):
    with rasterio.open(fname) as f:
        return f.affine

def get_poly_lines():
    with fiona.open('/Users/kirill/wk/dggs/qgis/data/cstauscd_l.shp') as f:
        return [np.r_[ pydash.get(p, 'geometry.coordinates') ].T 
                for p in f if pydash.get(p, 'properties.FEAT_CODE') == 'tile_edge']

In [ ]:
filename = '/Users/kirill/wk/dggs/abs_data/apg16e1_1_1_0.tif'

In [ ]:
A = get_affine(filename)
ds = xr.open_rasterio(filename)
ds.attrs['affine'] = A
#ds.values[ds.values<0] = 0

src = ds.isel(band=0)
ds.sum(), src.sum()

In [ ]:
np.unique( src.values[src.values<0].ravel() )

In [ ]:
plt.figure()
(src>0).plot.imshow()
plt.axis('equal')

In [ ]:
src.values.astype('float64').sum(), src.values.sum()

In [ ]:
W = src.values
iy,ix = np.where(W > 0)
vv = W[iy,ix]

#vv = src.values.ravel()
vv.min(), vv.max(), vv.sum(), vv.shape

In [ ]:
vv_ = spread_num_samples(vv)
vv_.sum(), vv.sum()

In [ ]:
plt.figure()
plt.hist( np.log(vv), 100)
pass

In [ ]:
plt.figure()
plt.imshow(np.isfinite(src.values))
h = plt.scatter(ix,iy, marker='.', s=np.log(vv+1.0), c='m')
h.set_alpha(0.1)

In [ ]:
crs = pyproj.Proj(ds.crs)

x_,y_ = A*(0.5 + np.vstack([ix,iy]))

plt.figure()
#src.plot.imshow()
plt.axis('equal')
h = plt.scatter(x_, y_, marker='.', s=np.log(vv+1.0), c='m')
h.set_alpha(0.1)

In [ ]:
#!gdalinfo /Users/kirill/wk/dggs/abs_data/apg16e1_1_1_0.tif

In [ ]:
lx, ly = crs(x_,y_, inverse=True)

plt.figure()
plt.plot(lx,ly,'.', markersize=0.1)
plt.axis('equal')
pass

In [ ]:
xy = gen_pts_from_distribution(src.values, A)
lx,ly = crs(xy[0], xy[1], inverse=True)

In [ ]:
plt.figure()
h = plt.plot(lx[::10], ly[::10], 'm.', markersize=0.1, alpha=0.3)
plt.axis('equal')

In [ ]:
#np.save('/Users/kirill/wk/dggs/abs_gen_pts.npy', np.c_[lx,ly])
#!ls -lh /Users/kirill/wk/dggs/abs_gen_pts.npy

In [ ]:
ds.values[0].shape